# Exploratory data analysis (EDA)

Use this notebook to explore and visualize the data.

In [ ]:
import ibis
import ibis.selectors as s

from icarus_investments.source import fake, seed_table
from icarus_investments.dag.config import DATA_DIR, RAW_DATA_DIR, BRONZE, SILVER, GOLD
from icarus_investments.catalog import Catalog

ibis.options.interactive = True

In [ ]:
seed_table

In [ ]:
catalog = Catalog()
catalog.list_groups()

In [ ]:
catalog.list_tables(GOLD)

In [ ]:
seed_table

In [ ]:
bs = ibis.read_parquet(f"{DATA_DIR}/{RAW_DATA_DIR}/buy_sell/*.parquet")
bs

In [ ]:
bs = catalog.table("buy_sell", BRONZE)
bs

In [ ]:
bs = catalog.table("buy_sell", SILVER)
bs

In [ ]:
ibis.random().name("random").as_table()

In [ ]:
bs.mutate(random=ibis.random())

In [ ]:
bs = catalog.table("buy_sell", GOLD)
bs

In [ ]:
bs.count()

In [ ]:
bs["location"].value_counts()

In [ ]:
f"{bs.count().to_pyarrow().as_py():,}"

In [ ]:
sm = ibis.read_parquet(f"{DATA_DIR}/{RAW_DATA_DIR}/social_media/*.parquet")
sm

In [ ]:
sm = catalog.table("social_media", SILVER)
sm